In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import numpy as np
from tqdm import tqdm
import dask.dataframe as dd

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Conv1D, MaxPool2D, Flatten, Dropout, CuDNNGRU, CuDNNLSTM, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, SGD, Nadam
from time import time
from livelossplot import PlotLossesKeras
from keras.layers.advanced_activations import LeakyReLU, PReLU
import tensorflow as tf
#from keras.utils.training_utils import multi_gpu_model
from tensorflow.python.client import device_lib
from sklearn.preprocessing import StandardScaler

from keijzer import *

import os
import numpy as np
import sys
import sklearn
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time
import matplotlib 
matplotlib.use('agg')
import matplotlib.pyplot as plt 
import pandas as pd

Using TensorFlow backend.


In [2]:
df = pd.read_csv("/home/roopesh/PythonFiles/Multivariate-time-series-models-in-Keras-master/data/house_data_processed.csv", delimiter='\t', parse_dates=['datetime'])
df = df.set_index(['datetime']) 

magnitude = 1 # Take this from the 1. EDA & Feauture engineering notebook. It's the factor where gasPower has been scaled with to the power 10.

In [3]:
df.head()

,FF,RG,T,gasPower,gasPower_std,hour,dayofweek,season
datetime,,,,,,,,
2017-02-28 12:00:00,9.067500,0.0,5.183333,6.115723,0.085626,12,1,1
2017-02-28 13:00:00,8.684999,0.0,5.300000,5.238647,0.318250,13,1,1
2017-02-28 14:00:00,8.296667,0.0,5.283333,2.451172,0.041087,14,1,1
2017-02-28 15:00:00,6.918334,0.0,5.516667,3.065186,0.214238,15,1,1
2017-02-28 16:00:00,6.583333,0.0,5.433333,8.120117,0.014156,16,1,1


In [4]:
data = df.copy()

In [5]:
columns_to_category = ['hour', 'dayofweek', 'season']
data[columns_to_category] = data[columns_to_category].astype('category') # change datetypes to category

In [6]:
data = pd.get_dummies(data, columns=columns_to_category) # One hot encoding the categories
data.head()

,FF,RG,T,gasPower,gasPower_std,hour_0,hour_1,hour_2,hour_3,hour_4,...,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,season_1,season_2,season_3,season_4
datetime,,,,,,,,,,,,,,,,,,,,,
2017-02-28 12:00:00,9.067500,0.0,5.183333,6.115723,0.085626,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2017-02-28 13:00:00,8.684999,0.0,5.300000,5.238647,0.318250,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2017-02-28 14:00:00,8.296667,0.0,5.283333,2.451172,0.041087,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2017-02-28 15:00:00,6.918334,0.0,5.516667,3.065186,0.214238,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2017-02-28 16:00:00,6.583333,0.0,5.433333,8.120117,0.014156,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [7]:
look_back = 5*24 # D -> 5, H -> 5*24
num_features = data.shape[1] - 1
output_dim = 1
train_size = 0.7

X_train, y_train, X_test, y_test = df_to_cnn_rnn_format(df=data, train_size=train_size, look_back=look_back, target_column='gasPower', scale_X=True)

In [8]:
X_train.shape

(4326, 120, 39)

In [9]:
plt.figure(figsize=(20,10))

plt.plot(data.index, data['gasPower'], '.-', color='red', label='Original data', alpha=0.5)
plt.xlabel('Datetime [-]', fontsize=20)
plt.ylabel(r'gasPower $\cdot$ 10$^{-%s}$ [m$^3$/h]' % (magnitude), fontsize=14)

plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)

plt.title('Mean gas usage of 52 houses', fontsize=14)

plt.tight_layout()


In [10]:
split_index = int(data.shape[0]*train_size)

X_train_values = data[:split_index] # get the datetime values of X_train
X_test_values = data[split_index:] # get the datetime values of X_train

In [11]:
X_train_values.shape, y_train.shape

((4446, 40), (4326,))

In [12]:
X_test_values.shape, y_test.shape

((1906, 40), (1786,))

In [13]:
datetime_difference = len(X_train_values) - len(y_train)
X_train_values = X_train_values[datetime_difference:] # Correct for datetime difference, this is a dirty way of doing it
X_train_values.shape, y_train.shape

datetime_difference = len(X_test_values) - len(y_test)
X_test_values = X_test_values[datetime_difference:] # Correct for datetime difference, this is a dirty way of doing it
X_test_values.shape, y_test.shape

((1786, 40), (1786,))

In [14]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_train.shape

(4326, 120, 39, 1)

In [17]:
def create_model(X_train, y_train, X_test, y_test):
    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    input_layer = keras.layers.Input(input_shape)
    # flatten/reshape because when multivariate all should be on the same axis 
    input_layer_flattened = keras.layers.Flatten()(input_layer)
    layer_1 = keras.layers.Dropout(0.1)(input_layer_flattened)
    layer_1 = keras.layers.Dense(500, activation='relu')(layer_1)
    layer_2 = keras.layers.Dropout(0.2)(layer_1)
    layer_2 = keras.layers.Dense(500, activation='relu')(layer_2)
    layer_3 = keras.layers.Dropout(0.2)(layer_2)
    layer_3 = keras.layers.Dense(500, activation='relu')(layer_3)
    output_layer = keras.layers.Dropout(0.3)(layer_3)
    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(output_layer)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])     
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=200, min_lr=0.1)
    file_path = '/home/roopesh/PythonFiles/Multivariate-time-series-models-in-Keras-master/best_model.hdf5' 
    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',save_best_only=True)
    
    return model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 39, 1)]      0         
_________________________________________________________________
flatten (Flatten)            (None, 4680)              0         
_________________________________________________________________
dropout (Dropout)            (None, 4680)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               2340500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               250500    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0     

In [16]:
model = create_model(X_train, y_train, X_test, y_test)

In [18]:
model.compile(optimizer='adam',loss='mse')

In [24]:
model.fit(X_train, y_train, epochs=epochs, batch_size=bs, validation_split=0.2,
         verbose=1,)

ValueError: Error when checking input: expected dense_3_input to have 2 dimensions, but got array with shape (4326, 120, 39, 1)